In [1]:
import tqdm
from tqdm import notebook

In [2]:
import os

In [3]:
import vault_client
yav = vault_client.instances.Production()

import pandas as pd
from databases import clickhouse as ch
from databases import mysql_db as msq
from databases import yql_db, chyt

import options
from options.get_vault_data import get_msql_password
from reports import generate_query

repeat = 1
class opts:
    type = 'day'
    date = '2021-11-06'
    datetime = date + ' 00:00:00'
    user = 'metr_analytics_ch_bot'
    passes = yav.get_version('ver-01f6q4fsg0t82b3byr22ametqs')
    password = passes['value']['metr_analytics_ch_bot']
    server = 'http://mtgiga001-1.metrika.yandex.net:8123/'
    test_server = 'http://mtgiga001-1t.metrika.yandex.net:8123/'
    yql_oauth = passes['value']['robot_yt_token']
    
def df_to_wiki(df, show_index = False):
    print('#|\n ||' + df.to_csv(index = show_index, sep = '|').replace('|', ' | ').replace('\n', '|| \n ||')[:-2] + '|#')    

In [59]:
start_timestamp = '2021-11-12 20:00:00'
end_timestamp = '2021-11-13 19:59:59'
utc_start_timestamp = '2021-11-12 20:00:00'
utc_end_timestamp = '2021-11-13 19:59:59'
release_time = '2021-11-12 20:00:00'

# start_timestamp = '2021-11-14 03:00:00'
# end_timestamp = '2021-11-15 02:59:59'
# utc_start_timestamp = '2021-11-14 03:00:00'
# utc_end_timestamp = '2021-11-15 02:59:59'
# release_time = '2021-11-14 03:00:00'

In [35]:
%%time 

q = '''
select 
    in_v2, in_v3,
    count() as visits
from
    (select 
        VisitID,
        1 as in_v2
    from merge.visits_v2 final 
    WHERE (StartDate >= toDate('{start_time}'))
        AND (StartDate <= toDate('{end_time}'))
        AND (StartTime >= '{start_time}')
        AND (StartTime <= '{end_time}')
        AND (UTCStartTime >= '{utc_start}')
        AND (UTCStartTime <= '{utc_end}')) as v2
    full join
    (select 
        VisitID,
        1 as in_v3
    from merge.visits_v3 final 
    WHERE (StartDate >= toDate('{start_time}'))
        AND (StartDate <= toDate('{end_time}'))
        AND (StartTime >= '{start_time}')
        AND (StartTime <= '{end_time}')
        AND (UTCStartTime >= '{utc_start}')
        AND (UTCStartTime <= '{utc_end}')) as v3
    using (VisitID)
group by in_v2, in_v3
format TSVWithNames
'''.format(start_time = start_timestamp, end_time = end_timestamp, 
           utc_start = utc_start_timestamp, utc_end = utc_end_timestamp)

print(q)

cmp_num_df = ch.get_df(q,  user = )

SyntaxError: invalid syntax (<unknown>, line 35)

In [21]:
cmp_num_df['volume'] = cmp_num_df.visits.map(lambda x: round(100.*x/cmp_num_df.visits.sum(), 2))

In [22]:
df_to_wiki(cmp_num_df.sort_values('visits', ascending = False))

#|
 ||in_v2 | in_v3 | visits | volume|| 
 ||1 | 0 | 4592643 | 92.89|| 
 ||1 | 1 | 351124 | 7.1|| 
 ||0 | 1 | 542 | 0.01|| 
 |#


# GET COLUMNS

In [5]:
q = '''describe merge.visits_v2 format TSVWithNames'''
visits_v2_columns_df = ch.get_df(q, user=opts.user, password=opts.password, host=opts.server)

In [6]:
q = '''describe merge.visits_v3 format TSVWithNames'''
visits_v3_columns_df = ch.get_df(q, user=opts.user, password=opts.password, host=opts.test_server)

In [7]:
visits_v2_columns_df[~visits_v2_columns_df.name.isin(visits_v3_columns_df.name.values)]

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
473,TrafficSource.ClickTrackID,Array(UInt64),NaN,NaN,NaN,NaN,NaN


In [8]:
visits_v3_columns_df[~visits_v3_columns_df.name.isin(visits_v2_columns_df.name.values)]

,name,type,default_type,default_expression,comment,codec_expression,ttl_expression
467,PrismPrior,UInt8,NaN,NaN,NaN,NaN,NaN
471,AreGoalsFraud,UInt8,NaN,NaN,NaN,NaN,NaN
472,IsFirstGoalsFraud,UInt8,NaN,NaN,NaN,NaN,NaN


In [9]:
common_columns = visits_v3_columns_df[visits_v3_columns_df.name.isin(visits_v2_columns_df.name.values)].name.values

In [10]:
len(common_columns)

476

# COMPARE

In [60]:
def process_column(start_timestamp, end_timestamp, col):
    q = '''
    select 
        '{col}' as column,
        count() as visits,
        countIf(v2_val = v2_test_val) as same_value_visits
    from
        (select 
            VisitID,
            TrafficSource.StartTime[indexOf(TrafficSource.Model, 3)] + (UTCStartTime - StartTime) as prod_first_utcstarttime,
            {col} as v2_val
        from merge.visits_v2 final 
        WHERE (StartDate >= toDate('{start_time}'))
            AND (StartDate <= toDate('{end_time}'))
            AND (StartTime >= '{start_time}')
            AND (StartTime <= '{end_time}')
            AND (UTCStartTime >= '{utc_start}')
            AND (UTCStartTime <= '{utc_end}')
            AND (dictGet('counters', 'mtlog_layer_id', toUInt64(CounterID)) = 1)) as v2
        inner join
        (select 
            VisitID,
            TrafficSource.StartTime[indexOf(TrafficSource.Model, 3)] + (UTCStartTime - StartTime) as test_first_utcstarttime,
            {col} as v2_test_val
        from remote('mtgiga001-1t.metrika.yandex.net', merge, visits_v2) final 
        WHERE (StartDate >= toDate('{start_time}'))
            AND (StartDate <= toDate('{end_time}'))
            AND (StartTime >= '{start_time}')
            AND (StartTime <= '{end_time}')
            AND (UTCStartTime >= '{utc_start}')
            AND (UTCStartTime <= '{utc_end}')
            AND (dictGet('counters', 'mtlog_layer_id', toUInt64(CounterID)) = 1)) as v3
    using (VisitID)
      Where 
        prod_first_utcstarttime >= '{release_time}' 
        and test_first_utcstarttime >= '{release_time}' 
    format TSVWithNames
    '''.format(start_time = start_timestamp, end_time = end_timestamp, col = col,
              utc_start = utc_start_timestamp, utc_end = utc_end_timestamp,
              release_time=release_time)
    return ch.get_df(q, user=opts.user, password=opts.password, host=opts.server)

In [61]:
! rm -rf data1112_2
! mkdir data1112_2

In [62]:
import os
loaded_cols = os.listdir('./data1112_2')
len(loaded_cols)

0

In [57]:
loaded_cols[:5]

[]

In [ ]:
tmp_dfs = []

for col in tqdm.notebook.tqdm(common_columns):
    if '%s.csv' % col.replace('.', '_') in loaded_cols:
        continue
    try:
        tmp_df = process_column(start_timestamp, end_timestamp, col)
        tmp_df.to_csv('./data1112_2/%s.csv' % col.replace('.', '_'), index = False, sep = '\t') 
        tmp_dfs.append(tmp_df)
    except:
        print(col, 'failed')

 Code: 160, e.displayText() = DB::Exception: Query is executing too slow: 824170.4271076049 rows/sec., minimum: 1000000: While executing Remote (version 21.8.11.4 (official build))



ParsedParams.Key1 failed


 Code: 160, e.displayText() = DB::Exception: Query is executing too slow: 784270.2195665503 rows/sec., minimum: 1000000: While executing Remote (version 21.8.11.4 (official build))



ParsedParams.Key2 failed


 Code: 160, e.displayText() = DB::Exception: Query is executing too slow: 798618.0319315079 rows/sec., minimum: 1000000: While executing Remote (version 21.8.11.4 (official build))



ParsedParams.Key3 failed


 Code: 160, e.displayText() = DB::Exception: Query is executing too slow: 797064.4662008877 rows/sec., minimum: 1000000: While executing Remote (version 21.8.11.4 (official build))



ParsedParams.Key4 failed


In [17]:
cols_df

NameError: name 'cols_df' is not defined

In [49]:
tmp_dfs = []

for filename in os.listdir('./data1'):
    tmp_dfs.append(pd.read_csv('./data1/' + filename, sep = '\t'))

In [61]:
pd.merge(cols_df_1, cols_df, on = 'column').to_csv('~/_outputs/compare_8_11-nov.csv')

In [50]:
cols_df = pd.concat(tmp_dfs)

In [51]:
cols_df['same_value_share'] = (100.*cols_df.same_value_visits/cols_df.visits).map(lambda x: round(x, 2))

In [52]:
df_to_wiki(cols_df.sort_values(['same_value_share', 'same_value_visits']))

#|
 ||column | visits | same_value_visits | same_value_share|| 
 ||AntiFraudInternalResult | 615422 | 0 | 0.0|| 
 ||RecountRequestID | 615422 | 0 | 0.0|| 
 ||VisitVersion | 615422 | 252967 | 41.1|| 
 ||IsNew | 615422 | 469175 | 76.24|| 
 ||TotalVisits | 615422 | 471765 | 76.66|| 
 ||TrafficSource.StartTime | 615422 | 495685 | 80.54|| 
 ||FirstVisit | 615422 | 543254 | 88.27|| 
 ||YAN.TurboType | 615422 | 549719 | 89.32|| 
 ||YAN.PageID | 615422 | 549719 | 89.32|| 
 ||YAN.URL | 615422 | 549719 | 89.32|| 
 ||YAN.AdSessionID | 615422 | 549719 | 89.32|| 
 ||YAN.DSPID | 615422 | 549719 | 89.32|| 
 ||YAN.BlockHeight | 615422 | 549719 | 89.32|| 
 ||YAN.PartnerPrice | 615422 | 549719 | 89.32|| 
 ||YAN.BlockWidth | 615422 | 549719 | 89.32|| 
 ||YAN.IsShow | 615422 | 549719 | 89.32|| 
 ||YAN.AdType | 615422 | 549719 | 89.32|| 
 ||YAN.AdbEnabled | 615422 | 549719 | 89.32|| 
 ||YAN.ImpID | 615422 | 549719 | 89.32|| 
 ||YAN.IsRender | 615422 | 549719 | 89.32|| 
 ||YAN.IsRequest | 615422 | 549719 | 

In [35]:
def diff_examples_column(start_timestamp, end_timestamp, col):
    q = '''
    select 
        VisitID, v2.CounterID, v2.UserID,
        '{col}' as column,
        v2_val,
        v2_val_test
    from
        (select 
            VisitID,CounterID, UserID,
            {col} as v2_val
        from merge.visits_v2 final sample 0.1
        WHERE (StartDate >= toDate('{start_time}'))
            AND (StartDate <= toDate('{end_time}'))
            AND (StartTime >= '{start_time}')
            AND (StartTime <= '{end_time}')
            AND (UTCStartTime >= '{utc_start}')
            AND (UTCStartTime <= '{utc_end}')
            AND (dictGet('counters', 'mtlog_layer_id', toUInt64(CounterID)) = 1)) as v2
        inner join
        (select 
            VisitID,
            {col} as v2_val_test
        from remote('mtgiga001-1t.metrika.yandex.net', merge, visits_v2) final sample 0.1
        WHERE (StartDate >= toDate('{start_time}'))
            AND (StartDate <= toDate('{end_time}'))
            AND (StartTime >= '{start_time}')
            AND (StartTime <= '{end_time}')
            AND (UTCStartTime >= '{utc_start}')
            AND (UTCStartTime <= '{utc_end}')
            AND (dictGet('counters', 'mtlog_layer_id', toUInt64(CounterID)) = 1)) as v3
        using (VisitID)
    where v2_val != v2_val_test 
    limit 10
    format TSVWithNames
    '''.format(start_time = start_timestamp, end_time = end_timestamp, col = col, 
               utc_start = utc_start_timestamp, utc_end = utc_end_timestamp)
    return ch.get_df(q, user=opts.user, password=opts.password, host=opts.server)

In [36]:
pd.set_option('display.max_colwidth', 5000)

In [37]:
tmp_df = diff_examples_column(start_timestamp, end_timestamp, '`TrafficSource.StartTime`')

In [41]:
from tabulate import tabulate
def pretty_df(df):
    print(tabulate(df, headers='keys', tablefmt='psql'))
    


In [38]:
df_to_wiki(tmp_df)

#|
 ||VisitID | CounterID | UserID | column | v2_val | v2_val_test|| 
 ||6768184424019001630 | 34 | 7190014551627817779 | `TrafficSource.StartTime` | ['2021-08-02 09:00:35','2021-11-14 22:49:34','2021-11-14 22:49:34','2021-11-14 22:49:34','2021-08-02 09:00:35','2021-11-14 22:49:34','2021-11-14 22:49:34'] | ['2021-08-02 09:00:38','2021-11-14 22:49:34','2021-11-14 22:49:34','2021-11-14 22:49:34','2021-08-02 09:00:38','2021-11-14 22:49:34','2021-11-14 22:49:34']|| 
 ||6767363250829918341 | 34 | 425836761630519376 | `TrafficSource.StartTime` | ['2021-09-01 21:23:46','2021-09-07 21:02:08','2021-09-07 21:02:08','2021-11-14 21:57:22','2020-12-28 22:32:36','2021-09-07 21:02:08','2021-09-07 21:02:08'] | ['2021-09-02 12:53:15','2021-11-14 21:57:22','2021-11-14 21:57:22','2021-11-14 21:57:22','2021-09-02 12:53:15','2021-11-14 21:57:22','2021-11-14 21:57:22']|| 
 ||6762730392714215545 | 34 | 2223092781621252239 | `TrafficSource.StartTime` | ['2021-05-17 15:12:58','2021-08-16 11:04:01','2021-08-16 

In [42]:
pretty_df(tmp_df)

+----+---------------------+-------------+---------------------+---------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------------------------------------------------------------------------------------------------------------------------------------------------+
|    |             VisitID |   CounterID |              UserID | column                    | v2_val                                                                                                                                                      | v2_val_test                                                                                                                                                 |
|----+---------------------+-------------+---------------------+---------------------------+------------------------------------------------------------------------------------------

# compare 2021-11-12 vs 2021-11-14

In [64]:
tmp_dfs_1112 = []

for filename in os.listdir('./data1112_2'):
    tmp_dfs_1112.append(pd.read_csv('./data1112_2/' + filename, sep = '\t'))
    
cols_df_1112 = pd.concat(tmp_dfs_1112)
cols_df_1112['same_value_share'] = (100.*cols_df_1112.same_value_visits/cols_df_1112.visits).map(lambda x: round(x, 2))
    

In [65]:
cols_df_1112

,column,visits,same_value_visits,same_value_share
0,EAction.ProductCategory4,312229,312195,99.99
0,UserAgent,312229,312227,100.00
0,Market.GoodQuantity,312229,312229,100.00
0,NetMinor,312229,312229,100.00
0,TotalVisits,312229,312198,99.99
...,...,...,...,...
0,CounterClass,312229,312229,100.00
0,YAN.IsRequest,312229,301517,96.57
0,IsPrivateMode,312229,312229,100.00
0,TrafficSource.UTMMedium,312229,310855,99.56


In [66]:
tmp_dfs_1114 = []

for filename in os.listdir('./data1114_2'):
    tmp_dfs_1114.append(pd.read_csv('./data1114_2/' + filename, sep = '\t'))
    
cols_df_1114 = pd.concat(tmp_dfs_1114)
cols_df_1114['same_value_share'] = (100.*cols_df_1114.same_value_visits/cols_df_1114.visits).map(lambda x: round(x, 2))
    

In [67]:
cols_df_1114[-cols_df_1114.column.isna()]

,column,visits,same_value_visits,same_value_share
0,EAction.ProductCategory4,329201,329167,99.99
0,UserAgent,329201,329200,100.00
0,Market.GoodQuantity,329201,329201,100.00
0,NetMinor,329201,329201,100.00
0,TotalVisits,329201,329190,100.00
...,...,...,...,...
0,CounterClass,329201,329201,100.00
0,YAN.IsRequest,329201,317265,96.37
0,IsPrivateMode,329201,329201,100.00
0,TrafficSource.UTMMedium,329201,327550,99.50


In [68]:
pd.merge(cols_df_1112, cols_df_1114, on = 'column').to_csv('~/_outputs/compare_12_14_2-nov.csv')